<a href="https://colab.research.google.com/github/andygomzang/Machine-Learning/blob/main/R2_A13_S14_Reinforcement_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Crear el laberinto de 40x40
def crear_laberinto():
    """
    Crea un laberinto de 40x40 con obstáculos aleatorios.
    - 0: Camino libre
    - 1: Obstáculos
    - 2: Meta
    """
    np.random.seed(0)  # Fijar semilla para reproducibilidad
    laberinto = np.zeros((40, 40), dtype=int)

    # Crear obstáculos aleatorios
    for _ in range(800):  # 800 obstáculos aleatorios
        x, y = np.random.randint(0, 40, size=2)
        laberinto[x, y] = 1

    # Asegurar inicio y meta
    laberinto[0, 0] = 0  # Inicio
    laberinto[39, 39] = 2  # Meta
    return laberinto

# Q-Learning
def entrenar_agente(laberinto, alpha=0.1, gamma=0.9, epsilon=0.1, episodios=60):
    """
    Entrena al agente utilizando el algoritmo de Q-Learning.
    """
    filas, columnas = laberinto.shape
    q_table = np.zeros((filas, columnas, 4))  # Q-Table inicializada (cuatro acciones)
    acciones = [(0, -1), (0, 1), (-1, 0), (1, 0)]  # Izquierda, derecha, arriba, abajo

    for episodio in range(episodios):
        estado = (0, 0)  # Inicio
        terminado = False

        while not terminado:
            x, y = estado

            # Selección de acción (exploración-explotación)
            if np.random.rand() < epsilon:
                accion_idx = np.random.randint(4)  # Exploración aleatoria
            else:
                accion_idx = np.argmax(q_table[x, y])  # Explotación (mejor acción)

            # Movimiento
            dx, dy = acciones[accion_idx]
            nuevo_x, nuevo_y = x + dx, y + dy

            # Validar movimiento
            if (
                0 <= nuevo_x < filas
                and 0 <= nuevo_y < columnas
                and laberinto[nuevo_x, nuevo_y] != 1
            ):
                nuevo_estado = (nuevo_x, nuevo_y)
            else:
                nuevo_estado = estado  # No mover si hay obstáculo

            # Recompensas
            if laberinto[nuevo_estado] == 2:  # Meta alcanzada
                recompensa = 100
                terminado = True
            else:
                recompensa = -1  # Penalización por cada paso

            # Actualizar Q-Table
            q_table[x, y, accion_idx] += alpha * (
                recompensa
                + gamma * np.max(q_table[nuevo_estado])
                - q_table[x, y, accion_idx]
            )

            estado = nuevo_estado

    return q_table

# Evaluar al Agente
def evaluar_agente(laberinto, q_table):
    """
    Evalúa el desempeño del agente mostrando el camino encontrado.
    """
    estado = (0, 0)
    camino = [estado]
    acciones = [(0, -1), (0, 1), (-1, 0), (1, 0)]  # Izquierda, derecha, arriba, abajo

    while laberinto[estado] != 2:
        x, y = estado
        accion_idx = np.argmax(q_table[x, y])
        dx, dy = acciones[accion_idx]
        nuevo_x, nuevo_y = x + dx, y + dy

        # Validar movimiento
        if (
            0 <= nuevo_x < laberinto.shape[0]
            and 0 <= nuevo_y < laberinto.shape[1]
            and laberinto[nuevo_x, nuevo_y] != 1
        ):
            estado = (nuevo_x, nuevo_y)
        camino.append(estado)

    return camino

# Visualizar el Camino
def mostrar_camino(laberinto, camino):
    """
    Muestra el laberinto y el camino recorrido por el agente.
    """
    laberinto_visual = laberinto.copy()
    for x, y in camino:
        if laberinto_visual[x, y] == 0:
            laberinto_visual[x, y] = 3  # Camino recorrido

    plt.figure(figsize=(8, 8))
    plt.imshow(laberinto_visual, cmap="viridis")
    plt.title("Camino Encontrado por el Agente")
    plt.colorbar()
    plt.show()

# Ejecución Principal
if __name__ == "__main__":
    # Crear laberinto
    laberinto = crear_laberinto()

    # Entrenar agente
    q_table = entrenar_agente(laberinto, alpha=0.1, gamma=0.9, epsilon=0.1, episodios=60)

    # Evaluar agente
    camino = evaluar_agente(laberinto, q_table)

    # Mostrar resultados
    mostrar_camino(laberinto, camino)
